In [1]:
from google.colab import files

uploaded = files.upload()


Saving archive.zip to archive.zip


In [2]:
import zipfile
import os

# Define the zip file name (replace 'yourfile.zip' with your actual file name)
zip_file = "archive.zip"

# Extract the contents
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall("extracted_folder")  # Extract to a specific folder

# List the extracted files
os.listdir("extracted_folder")


['Sarcasm_Headlines_Dataset.json', 'Sarcasm_Headlines_Dataset_v2.json']

In [3]:
!ls extracted_folder  # Lists the extracted files


Sarcasm_Headlines_Dataset.json	Sarcasm_Headlines_Dataset_v2.json


In [4]:
! pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which

In [5]:
import numpy as np
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict
from transformers import DataCollatorWithPadding, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments, AutoTokenizer, AutoConfig, AutoModel

import torch
import torch.nn as nn

import os
for dirname, _, filenames in os.walk('/content/extracted_folder'):
  for filename in filenames:
    print(os.path.join(dirname, filename))

/content/extracted_folder/Sarcasm_Headlines_Dataset.json
/content/extracted_folder/Sarcasm_Headlines_Dataset_v2.json


In [6]:
dataset_V2_path = '/content/extracted_folder/Sarcasm_Headlines_Dataset_v2.json'

In [7]:
df = pd.read_json(dataset_V2_path, lines=True)
df.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


# load dataset with HF load dataset

In [8]:
hf_dataset = load_dataset('json', data_files=dataset_V2_path)
hf_dataset.column_names

Generating train split: 0 examples [00:00, ? examples/s]

{'train': ['is_sarcastic', 'headline', 'article_link']}

In [9]:
hf_dataset = hf_dataset.remove_columns(['article_link'])
hf_dataset.set_format('pandas')
hf_dataset = hf_dataset['train'][:]
hf_dataset

,is_sarcastic,headline
0,1,thirtysomething scientists unveil doomsday clo...
1,0,dem rep. totally nails why congress is falling...
2,0,eat your veggies: 9 deliciously different recipes
3,1,inclement weather prevents liar from getting t...
4,1,mother comes pretty close to using word 'strea...
...,...,...
28614,1,jews to celebrate rosh hashasha or something
28615,1,internal affairs investigator disappointed con...
28616,0,the most beautiful acceptance speech this week...
28617,1,mars probe destroyed by orbiting spielberg-gat...


In [10]:
type(hf_dataset)


pandas.core.frame.DataFrame

In [11]:
hf_dataset.drop_duplicates(subset=['headline'], inplace=True)
hf_dataset = hf_dataset.reset_index()[['headline', 'is_sarcastic']]
hf_dataset = Dataset.from_pandas(hf_dataset)

train_testvalid = hf_dataset.train_test_split(test_size=0.2, seed=15)

test_valid = train_testvalid['test'].train_test_split(test_size=0.5, seed=15)

hf_dataset = DatasetDict({
    'train': train_testvalid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']
})

hf_dataset

DatasetDict({
    train: Dataset({
        features: ['headline', 'is_sarcastic'],
        num_rows: 22802
    })
    test: Dataset({
        features: ['headline', 'is_sarcastic'],
        num_rows: 2851
    })
    valid: Dataset({
        features: ['headline', 'is_sarcastic'],
        num_rows: 2850
    })
})

# Vector size "distilbert-base-uncased"
** In the model distilbert-base-uncased, each token is embedded into a vector of size 768. The shape of the output from the base model is **

# (batch_size, max_sequence_length, embedding_vector_size=768)

In [12]:
from transformers import AutoTokenizer

# Load the tokenizer (for example, BERT tokenizer)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

def tokenize(batch):
    return tokenizer(batch['headline'], truncation=True, max_length=512)

# Apply the tokenizer to the dataset
tokenized_dataset = hf_dataset.map(tokenize, batched=True)

print(tokenized_dataset)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/22802 [00:00<?, ? examples/s]

Map:   0%|          | 0/2851 [00:00<?, ? examples/s]

Map:   0%|          | 0/2850 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['headline', 'is_sarcastic', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 22802
    })
    test: Dataset({
        features: ['headline', 'is_sarcastic', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2851
    })
    valid: Dataset({
        features: ['headline', 'is_sarcastic', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2850
    })
})


In [13]:
tokenized_dataset = tokenized_dataset.remove_columns(['token_type_ids'])
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['headline', 'is_sarcastic', 'input_ids', 'attention_mask'],
        num_rows: 22802
    })
    test: Dataset({
        features: ['headline', 'is_sarcastic', 'input_ids', 'attention_mask'],
        num_rows: 2851
    })
    valid: Dataset({
        features: ['headline', 'is_sarcastic', 'input_ids', 'attention_mask'],
        num_rows: 2850
    })
})

In [14]:
# Rename 'is_sarcastic' to 'label'
tokenized_dataset = tokenized_dataset.rename_column("is_sarcastic", "label")

# Check the column names to confirm
print(tokenized_dataset.column_names)



{'train': ['headline', 'label', 'input_ids', 'attention_mask'], 'test': ['headline', 'label', 'input_ids', 'attention_mask'], 'valid': ['headline', 'label', 'input_ids', 'attention_mask']}


In [15]:
tokenized_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# We construct MyTaskSpecificCustomModel class that inherits from the nn.Module.

In [16]:
class MyTaskSpecificCustomModel(nn.Module):
  def __init__(self, checkpoint, num_labels):
    super(MyTaskSpecificCustomModel, self).__init__()

    self.num_labels = num_labels

    self.dropout = nn.Dropout(0.1)
    self.classifier = nn.Linear(768, num_labels)

  def forward(self, input_ids=None, attention_mask=None, labels=None):
    outputs = self.model(input_ids=input_ids, attention_mask=attentionmask)

    last_hidden_state = output[0]

    sequence_output = self.dropouts(last_hidden_state)

    logits = self.classifier(sequence_output[:, 0, :].view(-1, 768))

    loss = None
    if labels is not None:
      loss_fct = nn.CrossEntropyLoss()
      loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))

      return TokenClassifierOutput(loss=loss, logits=logits, hidden_states=outputs.hidden_states, attentions=outputs.attentions)


In [17]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(tokenized_dataset['train'], shuffle=True, batch_size=8, collate_fn=data_collator)
eval_dataloader = DataLoader(tokenized_dataset['valid'], batch_size=8, collate_fn=data_collator)

In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint = 'path_to_your_checkpoint_file_or_model'
model_task_specific = MyTaskSpecificCustomModel(checkpoint=checkpoint, num_labels=2).to(device)



In [26]:
from transformers import AdamW, get_scheduler


from transformers import AdamW, get_scheduler

optimizer = AdamW(model_task_specific.parameters(), lr = 5e-5 )

num_epoch = 3

num_warmup_steps = num_epoch * len(train_dataloader)

lr_scheduler = get_scheduler(
    'linear',
    optimizer=optimizer,
    num_warmup_steps=num_warmup_steps,  # Corrected argument
    num_training_steps=num_training_steps
)


In [27]:
! pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 3.4 MB/s eta 0:00:00


In [28]:
from evaluate import load
metric = load("f1")

In [30]:
from tqdm.auto import tqdm

progress_bar_train = tqdm(range(num_training_steps))
progress_bar_eval = tqdm(range(num_epoch * len(eval_dataloader)))

for epoch in range(num_epoch):  # Corrected this line
    model_task_specific.train()
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model_task_specific(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar_train.update(1)

    model_task_specific.eval()
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model_task_specific(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=batch['labels'])
        progress_bar_eval.update(1)

    print(metric.compute())


  0%|          | 0/8553 [00:00<?, ?it/s]

  0%|          | 0/1071 [00:00<?, ?it/s]

AttributeError: 'MyTaskSpecificCustomModel' object has no attribute 'model'

In [ ]:

model_task_specific.eval()

test_dataloader = DataLoader(
    tokenized_dataset['test'], batch_size = 32, collate_fn = data_collator
)


for batch in test_dataloader:
    batch = { k: v.to(device) for k, v in batch.items() }
    with torch.no_grad():
        outputs = model_task_specific(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim = -1)
    metric.add_batch(predictons = predictions, references=batch['labels'] )

metric.compute()
